In [1]:
import collections
import re
from d2l import torch as d2l

In [3]:
""" 加载文本数据集 """
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    """ 返回文本行列表 """
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'文本行数:{len(lines)}\neg:\n{lines[0]}\n{lines[10]}')

文本行数:3221
eg:
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [4]:
""" 词元化 """
def tokenize(lines, token='word'):
    """ 根据token参数将文本行拆分为char or word"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print(f'错误：未知token类型{token}')

tokens = tokenize(lines)    # 将所有文本行词元化
for i in range(10):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']


In [9]:
class Vocab:
    """ 构建词典 """
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        counter = count_corpus(tokens)  # 返回的是字典类型，key是token、val是出现次数
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)    # 对counter按出现次数重新降序排序

        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            else:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.to_tokens(ind) for ind in indices]

    @property
    def unk(self):
        return 0

    @property
    def token_freqs(self):
        """ 加了@property后，可以用调用属性的形式来调用方法,后面不需要加（）"""
        return self._token_freqs

def count_corpus(tokens):
    """ 统计词元出现频率 """
    if len(tokens) == 0 or isinstance(tokens[0], list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [13]:
vocab = Vocab(tokens)
list(vocab.token_to_idx.items())[:10]

[('<unk>', 0),
 ('the', 1),
 ('i', 2),
 ('and', 3),
 ('of', 4),
 ('a', 5),
 ('to', 6),
 ('was', 7),
 ('in', 8),
 ('that', 9)]

In [15]:
for i in [0, 10]:
    print(f'文本:\n{tokens[i]}\n{vocab[tokens[i]]}')

文本:
['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[1, 19, 50, 40, 2183, 2184, 400]
文本:
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
[2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [17]:
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, 'word')
    vocab = Vocab(tokens)

    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(32775, 4580)